In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

## Pandas Notebook Display Options

In [89]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 30

# Pandas Notes

> Data is separate from the index. 

Series is a single column, but with its own index, so printing it out will output two "columns". 

In [2]:
s = pd.Series(np.array([1, 3, 5, np.nan, 6, 8]))
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

## Creation

### Using CTOR

The ctor for `DataFrame` is `DataFrame(data, index, columns, dtype, copy)`. Of these only `data` is required. 

If column names and indexes are not specified, pandas will do its best to infer those. When it cannot, it will use a zero-based integer sequence as the index and the same for column names.

In [3]:
pd.DataFrame(np.random.randn(3, 5))

,0,1,2,3,4
0,1.270362,0.146177,0.095351,-0.116772,-0.065977
1,-0.349902,-1.200346,1.059452,1.135822,-1.402556
2,-0.399249,0.260114,-0.969537,2.258051,-0.316149


I can of course specify the column names and/or the index.

In [4]:
colnames = ["first", "second", "third", "fourth", "fifth"]
dates = pd.date_range('20190101', periods=3)
# dates = [datetime(2019, 1, 1), datetime(2019, 1, 2), datetime(2019, 1, 3)]
pd.DataFrame(np.random.randn(3, 5), columns=colnames, index=dates)

,first,second,third,fourth,fifth
2019-01-01,0.624969,1.778058,0.959490,-0.702746,-0.538507
2019-01-02,0.016360,-0.153971,2.210797,2.044499,1.092821
2019-01-03,-0.546008,0.340002,2.068481,-2.020662,0.109531


Here is an example where pandas does some clever inferring of the index and column names. The dict keys are used as column names. There is one column which has an index, that index is used as the index for the entire dataframe.

In [7]:
data = {
    "first": 1.,  # This single value will get copied through the col
    "second": pd.Timestamp("20190101"),  # So will this one
    "third": pd.Series(1, index=list(range(20, 35, 5)), dtype=float),  # This index will be used
    "fourth": pd.Categorical(["test", "train", "val"])
}
pd.DataFrame(data)

,first,second,third,fourth
20,1.0,2019-01-01,1.0,test
25,1.0,2019-01-01,1.0,train
30,1.0,2019-01-01,1.0,val


So what happens when two series have different indexes in the above example? 


It will try to merge the two indexes. And in doing so will try to replicate the other columns where they are empty. If the indexs have the same data type it is somewhat intuitive how the indexes are merged, as seen in the next cell. However, if the indexes have different types, then the merge becomes even more weird as can ben seen in the cell after next.

The upshot is pandas tries it best to create the dataframe.

In [8]:
data = {
    "first": 1.,
    "second": pd.Series(2, index=list(range(3))),
    "third": pd.Series(3, index=list(range(20, 35, 5)), dtype=float),
    "fourth": "test"
}
pd.DataFrame(data)

,first,second,third,fourth
0,1.0,2.0,NaN,test
1,1.0,2.0,NaN,test
2,1.0,2.0,NaN,test
20,1.0,NaN,3.0,test
25,1.0,NaN,3.0,test
30,1.0,NaN,3.0,test


Here the index data type is date and the second index is somehow forced into the date type with its values being taken as nano seconds (ugh!).

In [10]:
dates = pd.date_range('20190101', periods=3)
data = {
    "first": 1.,
    "second": pd.Series(2, index=dates),
    "third": pd.Series(3, index=list(range(20, 35, 5)), dtype=float),
    "fourth": "test"
}
pd.DataFrame(data)

,first,second,third,fourth
1970-01-01 00:00:00.000000020,1.0,NaN,NaN,test
1970-01-01 00:00:00.000000025,1.0,NaN,NaN,test
1970-01-01 00:00:00.000000030,1.0,NaN,NaN,test
2019-01-01 00:00:00.000000000,1.0,2.0,NaN,test
2019-01-02 00:00:00.000000000,1.0,2.0,NaN,test
2019-01-03 00:00:00.000000000,1.0,2.0,NaN,test


### From Files

Pandas supports file I/O for the following file formats -

  * CSV
  * JSON
  * HTML
  * Local clipboard
  * MS Excel
  * HDF5 Format
  * Feather
  * Parquet
  * Msgpack
  * Stata
  * SAS
  * Python pickle
  * SQL
  * Google Big Query
  
Below are some examples of creating dataframes from CSV files.

Lets load the titanic/train.csv file. This is a pretty well formatted CSV file so we can load it as-is. Pandas will infer the column names from the header row (which can also be specified) and will create a default index for the data.

In [11]:
pd.read_csv('./titanic/train.csv').head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We can also designate one of the columns in the CSV file as an index column.

In [12]:
pd.read_csv('./titanic/train.csv', index_col=0).head(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Even when reading files with datetime fields, pandas will still create a default index. Infact it will not even recognize the datetime column as a datetime. It will just be "object".

In [13]:
jena_climate = './jena_climate_2009_2016.csv'
df = pd.read_csv(jena_climate)

In [14]:
df.head(5)

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420551 entries, 0 to 420550
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date Time        420551 non-null  object 
 1   p (mbar)         420551 non-null  float64
 2   T (degC)         420551 non-null  float64
 3   Tpot (K)         420551 non-null  float64
 4   Tdew (degC)      420551 non-null  float64
 5   rh (%)           420551 non-null  float64
 6   VPmax (mbar)     420551 non-null  float64
 7   VPact (mbar)     420551 non-null  float64
 8   VPdef (mbar)     420551 non-null  float64
 9   sh (g/kg)        420551 non-null  float64
 10  H2OC (mmol/mol)  420551 non-null  float64
 11  rho (g/m**3)     420551 non-null  float64
 12  wv (m/s)         420551 non-null  float64
 13  max. wv (m/s)    420551 non-null  float64
 14  wd (deg)         420551 non-null  float64
dtypes: float64(14), object(1)
memory usage: 48.1+ MB


If I want pandas to recognize a column as a datetime column, I have to specify that. And if I want that column to be the index column, I have to specify that separately too.

In [16]:
df = pd.read_csv(jena_climate, parse_dates=["Date Time"], index_col="Date Time")

In [17]:
df.head(5)

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
2009-01-01 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2009-01-01 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
2009-01-01 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
2009-01-01 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 420551 entries, 2009-01-01 00:10:00 to 2017-01-01 00:00:00
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   p (mbar)         420551 non-null  float64
 1   T (degC)         420551 non-null  float64
 2   Tpot (K)         420551 non-null  float64
 3   Tdew (degC)      420551 non-null  float64
 4   rh (%)           420551 non-null  float64
 5   VPmax (mbar)     420551 non-null  float64
 6   VPact (mbar)     420551 non-null  float64
 7   VPdef (mbar)     420551 non-null  float64
 8   sh (g/kg)        420551 non-null  float64
 9   H2OC (mmol/mol)  420551 non-null  float64
 10  rho (g/m**3)     420551 non-null  float64
 11  wv (m/s)         420551 non-null  float64
 12  max. wv (m/s)    420551 non-null  float64
 13  wd (deg)         420551 non-null  float64
dtypes: float64(14)
memory usage: 48.1 MB


## Common Functions

In general avoid `DataFrame.to_numpy()`. It only works properly if all the columns have the same data type. Otherwise pandas will convert everything to the lowest common dtype, which is usually object. 


Another weirdness about this method is that it will not include the index column. Notice that in the original df there was a date based index, which is missing in the numpy array.

In [19]:
dates = pd.date_range('20190101', periods=6)
data = np.random.randn(6, 6)
df = pd.DataFrame(data=data, index=dates)
df

,0,1,2,3,4,5
2019-01-01,-0.271887,1.436796,1.098205,-0.267391,1.930223,-0.370065
2019-01-02,-0.947612,-0.444200,-0.962933,-1.176548,0.810035,1.586071
2019-01-03,-0.314878,-1.946615,-1.168064,-0.377930,1.124294,-0.292441
2019-01-04,-0.636162,1.437700,-0.277411,-0.967682,0.008286,0.171578
2019-01-05,0.528936,-1.167646,-0.992968,-1.873845,0.390955,-1.587635
2019-01-06,-0.535603,-0.398965,-1.732721,0.208501,0.060473,-0.788893


In [20]:
df.to_numpy()

array([[-0.27188692,  1.43679631,  1.09820471, -0.26739111,  1.93022252,
        -0.37006488],
       [-0.94761174, -0.44419979, -0.96293335, -1.17654823,  0.81003548,
         1.58607127],
       [-0.31487827, -1.9466152 , -1.16806431, -0.37792981,  1.12429388,
        -0.2924412 ],
       [-0.63616152,  1.43769996, -0.27741067, -0.96768229,  0.00828596,
         0.17157795],
       [ 0.52893594, -1.16764551, -0.99296782, -1.87384503,  0.39095456,
        -1.58763486],
       [-0.53560263, -0.39896498, -1.73272052,  0.20850068,  0.06047272,
        -0.78889308]])

In [21]:
first_col = np.arange(5)
np.random.shuffle(first_col)
second_col = np.array(["apple", "banana", "cantaloupe", "dates", "guava"])
np.random.shuffle(second_col)
index_col = np.array([
    datetime(2019, 1, 1, 10),
    datetime(2019, 1, 1, 11),
    datetime(2019, 1, 2, 3, 30),
    datetime(2019, 1, 2, 4, 45),
    datetime(2019, 1, 3)
])
np.random.shuffle(index_col)
data = {
    "first_col": first_col,
    "second_col": second_col
}
df = pd.DataFrame(data=data, index=index_col)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2019-01-01 11:00:00 to 2019-01-03 00:00:00
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   first_col   5 non-null      int64 
 1   second_col  5 non-null      object
dtypes: int64(1), object(1)
memory usage: 120.0+ bytes


In [23]:
df.describe()

,first_col
count,5.000000
mean,2.000000
std,1.581139
min,0.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,4.000000


In [24]:
df.index

DatetimeIndex(['2019-01-01 11:00:00', '2019-01-01 10:00:00',
               '2019-01-02 03:30:00', '2019-01-02 04:45:00',
               '2019-01-03 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [25]:
df.shape

(5, 2)

In [26]:
df.head(2)

,first_col,second_col
2019-01-01 11:00:00,1,banana
2019-01-01 10:00:00,4,cantaloupe


In [27]:
df.tail(2)

,first_col,second_col
2019-01-02 04:45:00,2,apple
2019-01-03 00:00:00,0,dates


In [28]:
df.T

,2019-01-01 11:00:00,2019-01-01 10:00:00,2019-01-02 03:30:00,2019-01-02 04:45:00,2019-01-03 00:00:00
first_col,1,4,3,2,0
second_col,banana,cantaloupe,guava,apple,dates


Axes pretty much have the same meaning that they do in numpy. It is simpler in pandas because pandas dataframes are always 2-dim, so there are only two axis. Axis=0 corresponds to columns and axis=1 corresponds to rows.

The section on sorting by axis makes this even more clear.

In [29]:
df.axes

[DatetimeIndex(['2019-01-01 11:00:00', '2019-01-01 10:00:00',
                '2019-01-02 03:30:00', '2019-01-02 04:45:00',
                '2019-01-03 00:00:00'],
               dtype='datetime64[ns]', freq=None),
 Index(['first_col', 'second_col'], dtype='object')]

## Missing Values
Null values can be detected by one of two functions - `isna()` and `isnull()`. To return all rows that have at least one cell as NA run the following query. This technique can be used for any boolean condition.

For a detailed walkthrough of various missing value scenarios see [the user guide](https://pandas.pydata.org/docs/user_guide/missing_data.html#inserting-missing-data)

In [71]:
df[df.isna().any(axis=1)]

,first,second,third
1,NaN,0.128176,0.095962
3,0.45587,NaN,0.720207
4,NaN,0.798483,0.684073


### Drop Missing Values
Lets create a dataframe with staggered missing values. The simple `dropna` API will drop any row that has even a single missing cell. It will return a new dataframe with only full rows.

In [72]:
rng = np.random.default_rng()

In [73]:
np.set_printoptions(
    edgeitems=30, 
    linewidth=100000, 
    precision=3,
    suppress=True)

In [74]:
data = rng.random((5, 3))
data[1, 0] = None
data[4, 0] = None
data[3, 1] = None
data

array([[0.113, 0.4  , 0.851],
       [  nan, 0.034, 0.793],
       [0.113, 0.159, 0.813],
       [0.978,   nan, 0.368],
       [  nan, 0.241, 0.879]])

In [75]:
colnames = ["first", "second", "third"]
df = pd.DataFrame(data, columns=colnames)
df

,first,second,third
0,0.112819,0.399560,0.851414
1,NaN,0.033837,0.793316
2,0.113165,0.158880,0.813207
3,0.977968,NaN,0.367621
4,NaN,0.241207,0.878955


In [78]:
# dfs are immutable, so this op just creates a new df with all full rows.
df.dropna()

,first,second,third
0,0.112819,0.39956,0.851414
2,0.113165,0.15888,0.813207


In [80]:
# The original df is still intact
df

,first,second,third
0,0.112819,0.399560,0.851414
1,NaN,0.033837,0.793316
2,0.113165,0.158880,0.813207
3,0.977968,NaN,0.367621
4,NaN,0.241207,0.878955


### Replace Missing Values
Another thing I might want to do is to replace the missing values with some other values on a per column basis. Let me replace the missing values in the `first` column with `0` and the missing values in the `second` column with the average of the column.

In [84]:
df["full_first"] = df["first"].fillna(0)
df

,first,second,third,full_first
0,0.112819,0.399560,0.851414,0.112819
1,NaN,0.033837,0.793316,0.000000
2,0.113165,0.158880,0.813207,0.113165
3,0.977968,NaN,0.367621,0.977968
4,NaN,0.241207,0.878955,0.000000


In [87]:
fill_val = df["second"].mean()
fill_val

0.2083710935378544

In [88]:
df["full_second"] = df["second"].fillna(fill_val)
df

,first,second,third,full_first,full_second
0,0.112819,0.399560,0.851414,0.112819,0.399560
1,NaN,0.033837,0.793316,0.000000,0.033837
2,0.113165,0.158880,0.813207,0.113165,0.158880
3,0.977968,NaN,0.367621,0.977968,0.208371
4,NaN,0.241207,0.878955,0.000000,0.241207


## Cardinality
To get the cardinality of all the columns in a single query

In [32]:
df.apply(pd.Series.nunique)

first_col     5
second_col    5
dtype: int64

## Sorting

`df.sort_index()` does what is expected, it will sort by the index column.

In [33]:
df.sort_index()

,first_col,second_col
2019-01-01 10:00:00,4,cantaloupe
2019-01-01 11:00:00,1,banana
2019-01-02 03:30:00,3,guava
2019-01-02 04:45:00,2,apple
2019-01-03 00:00:00,0,dates


And so does `df.sort_values(by=colname)`.

In [34]:
df.sort_values(by="first_col")

,first_col,second_col
2019-01-03 00:00:00,0,dates
2019-01-01 11:00:00,1,banana
2019-01-02 04:45:00,2,apple
2019-01-02 03:30:00,3,guava
2019-01-01 10:00:00,4,cantaloupe


I can ask pandas to sort by the column names. This is where the concept of axis comes in. For that lets create a new dataframe.

In [35]:
data = np.arange(15)
np.random.shuffle(data)
data = data.reshape(5, 3)
data

array([[ 0,  1,  6],
       [ 7,  5, 14],
       [ 9, 11,  2],
       [ 4,  8, 12],
       [13, 10,  3]])

In [36]:
colnames = ["C", "A", "B"]
df = pd.DataFrame(data, columns=colnames)
df

,C,A,B
0,0,1,6
1,7,5,14
2,9,11,2
3,4,8,12
4,13,10,3


In [37]:
df.sort_index(axis=1)

,A,B,C
0,1,6,0
1,5,14,7
2,11,2,9
3,8,12,4
4,10,3,13


## SQL Equivalence

Really [good tutorial](http://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html) on pandas website.

In [38]:
tips = pd.read_csv("./tips.csv")

In [39]:
tips.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [40]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


### Select Queries

```sql
SELECT sex, smoker
FROM tips
WHERE time = 'Dinner' AND tip > 5.00
```

The process is simple -

  1. First create the individual query masks
  2. Combine the individual query masks into a single query mask
  3. Get the data
  
Null checking masks are created using the `isna()` and `notna()` methods.  

In [41]:
is_dinner = tips["time"] == "Dinner"
is_dinner[:5]

0    True
1    True
2    True
3    True
4    True
Name: time, dtype: bool

In [42]:
big_tip = tips["tip"] > 5.
big_tip[:5]

0    False
1    False
2    False
3    False
4    False
Name: tip, dtype: bool

In [43]:
query = is_dinner & big_tip
query[:5]

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [44]:
tips[query]
# tips[(tips.tip > 5.) & (tips.time == "Dinner")]["size", "smoker"]

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
116,29.93,5.07,Male,No,Sun,Dinner,4
155,29.85,5.14,Female,No,Sun,Dinner,5
170,50.81,10.00,Male,Yes,Sat,Dinner,3
172,7.25,5.15,Male,Yes,Sun,Dinner,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2


### Group By

Simplest group by query is to find out how many males and how many females are in the dataframe.

```sql
SELECT sex, COUNT(*)
FROM tips
GROUP BY sex;
```

In pandas the `size()` function is used instead of `count()`. The latter will give the non-null entries in each column. The return type will be a Series or DataFrame depending on how many columns are being retruned. The grouped column will be the index.

In [45]:
x = tips.groupby("sex").size()
print(type(x))

<class 'pandas.core.series.Series'>


In [46]:
x

sex
Female     87
Male      157
dtype: int64

In [47]:
x = tips.groupby("sex").count()
print(type(x))

<class 'pandas.core.frame.DataFrame'>


In [48]:
x

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,87,87,87,87,87,87
Male,157,157,157,157,157,157


It is possible to aggregate different columns differently. Just use the `agg()` function passing it a dict of column names and the function pointer that will aggregate the column vector.

It is also possible to group by multiple columns. This will create a DataFrame with multiindex.

```sql
SELECT smoker, day, COUNT(*), AVG(tip)
FROM tips
GROUP BY smoker, day
```

In the example below I chose to get the size of the day column because all columns have equal size.

In [49]:
df = tips.groupby(["smoker", "day"]).agg({"tip": np.mean, "day": np.size})

In [50]:
df

tip  day
smoker day                
No     Fri   2.812500    4
       Sat   3.102889   45
       Sun   3.167895   57
       Thur  2.673778   45
Yes    Fri   2.714000   15
       Sat   2.875476   42
       Sun   3.516842   19
       Thur  3.030000   17

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8 entries, ('No', 'Fri') to ('Yes', 'Thur')
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tip     8 non-null      float64
 1   day     8 non-null      int64  
dtypes: float64(1), int64(1)
memory usage: 299.0+ bytes


### Updates

```sql
UPDATE tips
SET tip = tip*2
WHERE tip < 2
```

In [52]:
tips[:5]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [53]:
filter_mask = tips["tip"] < 2
col_to_update = "tip"
tips.loc[filter_mask, col_to_update] *= 2
tips[:5]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,2.02,Female,No,Sun,Dinner,2
1,10.34,3.32,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### Deletes

Just select the rows that I want to keep and assing to the same dataframe variable.

```sql
DELETE FROM tips
WHERE tip > 9
```

In [54]:
tips.shape

(244, 7)

In [55]:
inverse_filter_mask = tips["tip"] <= 9
tips = tips.loc[inverse_filter_mask]
tips.shape

(243, 7)

### Joins

In order to illustrate joins let us create a simple RDBMS mapping

users --> favorites <-- websites

In [56]:
user_ids = [1, 2, 3]
user_details = {
    "name": ["Happy Orange", "Cookie Monster", "Frozen Horizon"],
    "email": ["happy@orange.com", "cookie@monster.com", "frozen@horizon.com"],
    "created_at": [pd.Timestamp("20150101"), pd.Timestamp("20150203"), pd.Timestamp("20160201")]
}
users = pd.DataFrame(data=user_details, index=user_ids)
users

,name,email,created_at
1,Happy Orange,happy@orange.com,2015-01-01
2,Cookie Monster,cookie@monster.com,2015-02-03
3,Frozen Horizon,frozen@horizon.com,2016-02-01


In [57]:
website_ids = [1, 2, 3, 4, 5]
titles = [
    "Gigaom",
    "Techcrunch",
    "Hacker News",
    "Overview -- Python",
    "Quanta Magazine"
]
urls = [
    "https://gigaom.com",
    "http://techcrunch.com",
    "https://news.ycombinator.com",
    "https://docs.python.org",
    "http://simonsfoundation.org/quanta"
]
crawled_at = [
    pd.Timestamp("2015-09-16 21:18:23"),
    pd.Timestamp("2015-09-16 21:19:23"),
    pd.Timestamp("2014-09-16 21:19:23"),
    pd.Timestamp("2014-09-16 21:19:23"),
    pd.Timestamp("2014-09-16 21:19:23")
]
website_details = {
    "title": titles,
    "url": urls,
    "crawled_at": crawled_at
}
websites = pd.DataFrame(data=website_details, index=website_ids)
websites

,title,url,crawled_at
1,Gigaom,https://gigaom.com,2015-09-16 21:18:23
2,Techcrunch,http://techcrunch.com,2015-09-16 21:19:23
3,Hacker News,https://news.ycombinator.com,2014-09-16 21:19:23
4,Overview -- Python,https://docs.python.org,2014-09-16 21:19:23
5,Quanta Magazine,http://simonsfoundation.org/quanta,2014-09-16 21:19:23


In [58]:
added_ons = [
    pd.Timestamp("2015-09-16 21:19:23"),
    pd.Timestamp("2015-09-16 21:19:23"),
    pd.Timestamp("2015-09-16 21:19:23"),
    pd.Timestamp("2015-09-16 21:19:23"),
    pd.Timestamp("2015-09-16 21:19:23"),
    pd.Timestamp("2015-09-16 21:19:23"),
    pd.Timestamp("2015-09-16 21:19:23"),
]
names = [
    "Giga Om Biz 2.0",
    "Tech Gossip",
    "H@ck3rz",
    "techy love",
    "YC news",
    "python reference",
    "Science news"
]
fav_details = {
    "user_id": [1, 1, 1, 2, 2, 2, 3],
    "website_id": [1, 2, 3, 2, 3, 4, 5],
    "added_on":added_ons,
    "name": names
}

favorites = pd.DataFrame(data=fav_details)
favorites

,user_id,website_id,added_on,name
0,1,1,2015-09-16 21:19:23,Giga Om Biz 2.0
1,1,2,2015-09-16 21:19:23,Tech Gossip
2,1,3,2015-09-16 21:19:23,H@ck3rz
3,2,2,2015-09-16 21:19:23,techy love
4,2,3,2015-09-16 21:19:23,YC news
5,2,4,2015-09-16 21:19:23,python reference
6,3,5,2015-09-16 21:19:23,Science news


```sql
SELECT *
FROM users
INNER JOIN favorites
ON users.index_col = favorites.user_id
```

The users dataframe index is not called "user_id". In order to join users and favorites tables on the user id, I'll have to join the (un-named) index column in users with the user_id column in favorites.

In [59]:
users.merge(favorites, left_index=True, right_on="user_id")

,name_x,email,created_at,user_id,website_id,added_on,name_y
0,Happy Orange,happy@orange.com,2015-01-01,1,1,2015-09-16 21:19:23,Giga Om Biz 2.0
1,Happy Orange,happy@orange.com,2015-01-01,1,2,2015-09-16 21:19:23,Tech Gossip
2,Happy Orange,happy@orange.com,2015-01-01,1,3,2015-09-16 21:19:23,H@ck3rz
3,Cookie Monster,cookie@monster.com,2015-02-03,2,2,2015-09-16 21:19:23,techy love
4,Cookie Monster,cookie@monster.com,2015-02-03,2,3,2015-09-16 21:19:23,YC news
5,Cookie Monster,cookie@monster.com,2015-02-03,2,4,2015-09-16 21:19:23,python reference
6,Frozen Horizon,frozen@horizon.com,2016-02-01,3,5,2015-09-16 21:19:23,Science news


I can also join on common columns, i.e., they must have the same column names in both the dataframes using the `on` method argument.

## Merging

TODO: Go through http://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Iterating

In [60]:
for idx, row in tips.iterrows():
    print(idx, row.total_bill, row.tip, row.sex, row.smoker)

0 16.99 2.02 Female No
1 10.34 3.32 Male No
2 21.01 3.5 Male No
3 23.68 3.31 Male No
4 24.59 3.61 Female No
5 25.29 4.71 Male No
6 8.77 2.0 Male No
7 26.88 3.12 Male No
8 15.04 3.92 Male No
9 14.78 3.23 Male No
10 10.27 3.42 Male No
11 35.26 5.0 Female No
12 15.42 3.14 Male No
13 18.43 3.0 Male No
14 14.83 3.02 Female No
15 21.58 3.92 Male No
16 10.33 3.34 Female No
17 16.29 3.71 Male No
18 16.97 3.5 Female No
19 20.65 3.35 Male No
20 17.92 4.08 Male No
21 20.29 2.75 Female No
22 15.77 2.23 Female No
23 39.42 7.58 Male No
24 19.82 3.18 Male No
25 17.81 2.34 Male No
26 13.37 2.0 Male No
27 12.69 2.0 Male No
28 21.7 4.3 Male No
29 19.65 3.0 Female No
30 9.55 2.9 Male No
31 18.35 2.5 Male No
32 15.06 3.0 Female No
33 20.69 2.45 Female No
34 17.78 3.27 Male No
35 24.06 3.6 Male No
36 16.31 2.0 Male No
37 16.93 3.07 Female No
38 18.69 2.31 Male No
39 31.27 5.0 Male No
40 16.04 2.24 Male No
41 17.46 2.54 Male No
42 13.94 3.06 Male No
43 9.68 2.64 Male No
44 30.4 5.6 Male No
45 18.29 3.0 Male